In [ ]:
from main import *

# toy example

train_sub = [1]
sigma = 500
train_x0, train_y0 = make_train_dataset(train_sub, 0)
score_model0 = train_scorenet(train_x0, train_y0, sigma)

Score model Average Loss: 676.630907, lr : 0.0002503155504993245:  16%|███████████████████████▌                                                                                                                        | 490/3000 [13:19<1:08:07,  1.63s/it]

In [ ]:
batch_size = 1
samples0 = sampling(score_model0, sample_batch_size=batch_size)

generated_signal0 = return_to_signal(samples0)

In [ ]:
import matplotlib.pyplot as plt


for sample in samples0:
    plt.plot(sample[0][0].cpu())
    plt.show()

for signal in generated_signal0[0][0]:
    plt.plot(signal)
    plt.show()


for x in train_x0:
    plt.plot(x[0])
    plt.show()
    
